# Pyspark Basics

[docs](https://docs.databricks.com/aws/en/pyspark/basics)
ใน part นี้ คำอธิบายและ code จะอยู่ในไฟล์นี้ไฟล์เดียว

## Import data types

In [0]:
# import select functions and types
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import floor, round

# import modules using an alias
import pyspark.sql.types as T
import pyspark.sql.functions as F

## Create a DataFrame

### Create a DataFrame with specified values

In [0]:
df_children = spark.createDataFrame(
  data = [("Mikhail", 15), ("Zaky", 13), ("Zoya", 8)],
  schema = ['name', 'age'])
display(df_children)

(อันนี้แบบระบุ Data Type 👇)

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

df_children_with_schema = spark.createDataFrame(
  data = [("Mikhail", 15), ("Zaky", 13), ("Zoya", 8)],
  schema = StructType([
    StructField('name', StringType(), True),
    StructField('age', IntegerType(), True)
  ])
)
display(df_children_with_schema)

### Create a DataFrame from a table in Unity Catalog

In [0]:
df_customer = spark.table('samples.tpch.customer')
display(df_customer)

### Create a DataFrame from an uploaded file

In [0]:
# Assign this variable your full volume file path
volume_file_path = "/Volumes/workspace/default/tutorial/rows.csv"

df_csv = (spark.read
  .format("csv")
  .option("header", True)
  .option("inferSchema", True)
  .load(volume_file_path)
)
display(df_csv)

### Create a DataFrame from a JSON response
(ติดไว้ก่อน)

In [0]:
import requests

# Download data from URL
url = "https://dummyjson.com/products"
response = requests.get(url)

In [0]:
products = response.json()["products"]

for p in products:
    p["price"] = float(p["price"])
    p["rating"] = float(p["rating"])
    p["discountPercentage"] = float(p["discountPercentage"])

df_json = spark.createDataFrame(products)
display(df_json)


#### Select a JSON field or object 

In [0]:
display(df_json.select(df_json["reviews"]))

In [0]:
display(df_json.select(
        df_json["reviews"][1]["reviewerName"],
        df_json["reviews"][1]["rating"],
        df_json["reviews"][1]["comment"]
    ))

### Create a DataFrame from a file

In [0]:
display(dbutils.fs.ls('/databricks-datasets'))

In [0]:
%fs ls '/databricks-datasets'

In [0]:
df_population = (spark.read
  .format("csv")
  .option("header", True)
  .option("inferSchema", True)
  .load("/databricks-datasets/samples/population-vs-price/data_geo.csv")
)
display(df_population)

## Transform data with DataFrames

### Column operations

#### Select columns

In [0]:
from pyspark.sql.functions import col

df_customer.select(
  col("c_custkey"),
  col("c_acctbal")
)

In [0]:
from pyspark.sql.functions import expr

df_customer.select(
  expr("c_custkey"),
  expr("c_acctbal")
)

In [0]:
df_customer.selectExpr(
  "c_custkey as key",
  "round(c_acctbal) as account_rounded"
)

In [0]:
df_customer.select(
  "c_custkey",
  "c_acctbal"
)

In [0]:
df_customer.select(
  df_customer["c_custkey"],
  df_customer["c_acctbal"]
)

In [0]:
df_customer.select(
  df_customer.c_custkey,
  df_customer.c_acctbal
)

#### Create columns

In [0]:
df_customer_flag = df_customer.withColumn("balance_flag", col("c_acctbal") > 1000)

In [0]:
display(df_customer_flag)

#### Rename columns

In [0]:
df_customer_flag_renamed = df_customer_flag.withColumnRenamed("balance_flag", "balance_flag_renamed")

In [0]:
display(df_customer_flag_renamed)

In [0]:
from pyspark.sql.functions import avg

df_segment_balance = df_customer.groupBy("c_mktsegment").agg(
    avg(df_customer["c_acctbal"]).alias("avg_account_balance")
)

display(df_segment_balance)

#### Cast column types

In [0]:
from pyspark.sql.functions import col

df_casted = df_customer.withColumn("c_custkey", col("c_custkey").cast(StringType()))
print(type(df_casted))

#### Remove columns

In [0]:
df_customer_flag_renamed.drop("balance_flag_renamed")

In [0]:
df_customer_flag_renamed.drop("c_phone", "balance_flag_renamed")

### Row operations

#### Filter rows

In [0]:
from pyspark.sql.functions import col

df_that_one_customer = df_customer.filter(col("c_custkey") == 412449)

In [0]:
display(df_customer.filter((col("c_nationkey") == 20) & (col("c_acctbal") > 1000)))

In [0]:
df_filtered_customer = df_customer.filter((col("c_custkey") == 412446) | (col("c_custkey") == 412447))

In [0]:
display(df_filtered_customer)

#### Remove duplicate rows

In [0]:
df_unique = df_customer.distinct()

#### Handle null values

In [0]:
df_customer_no_nulls = df_customer.na.drop()
df_customer_no_nulls = df_customer.na.drop("any")

In [0]:
df_customer_no_nulls = df_customer.na.drop("all")

In [0]:
df_customer_no_nulls = df_customer.na.drop("all", subset=["c_acctbal", "c_custkey"])

In [0]:
df_customer_filled = df_customer.na.fill("0", subset=["c_acctbal"])

In [0]:
df_customer_phone_filled = df_customer.na.replace([""], ["UNKNOWN"], subset=["c_phone"])

#### Append rows

In [0]:
df_appended_rows = df_that_one_customer.union(df_filtered_customer)

display(df_appended_rows)

#### Sort rows

In [0]:
display(df_customer.orderBy(col("c_acctbal")))

In [0]:
display(df_customer.sort(col("c_custkey").desc()))

In [0]:
df_sorted = df_customer.orderBy(col("c_acctbal").desc(), col("c_custkey").asc())
df_sorted = df_customer.sort(col("c_acctbal").desc(), col("c_custkey").asc())

In [0]:
display(df_sorted.limit(10))

### Join DataFrames

In [0]:
df_customer = spark.table('samples.tpch.customer')
df_order = spark.table('samples.tpch.orders')

df_joined = df_order.join(
  df_customer,
  on = df_order["o_custkey"] == df_customer["c_custkey"],
  how = "inner"
)

display(df_joined)

In [0]:
df_customer = spark.table('samples.tpch.customer')
df_order = spark.table('samples.tpch.orders')

df_complex_joined = df_order.join(
  df_customer,
  on = ((df_order["o_custkey"] == df_customer["c_custkey"]) & (df_order["o_totalprice"] > 500000)),
  how = "inner"
)

display(df_complex_joined)

### Aggregate data

In [0]:
from pyspark.sql.functions import avg

# group by one column
df_segment_balance = df_customer.groupBy("c_mktsegment").agg(
    avg(df_customer["c_acctbal"])
)

display(df_segment_balance)

In [0]:
from pyspark.sql.functions import avg

# group by two columns
df_segment_nation_balance = df_customer.groupBy("c_mktsegment", "c_nationkey").agg(
    avg(df_customer["c_acctbal"])
)

display(df_segment_nation_balance)

In [0]:
df_customer.count()

### Chaining calls

In [0]:
from pyspark.sql.functions import count

df_chained = (
    df_order.filter(col("o_orderstatus") == "F")
    .groupBy(col("o_orderpriority"))
    .agg(count(col("o_orderkey")).alias("n_orders"))
    .sort(col("n_orders").desc())
)

display(df_chained)

## Visualize your DataFrame

In [0]:
display(df_order)

## Save your data

### Save your DataFrame as a table

In [0]:
df_joined.write.saveAsTable(f"workspace.default.customer_info")

### Write your DataFrame as CSV

In [0]:
# Assign this variable your file path
file_path = "/Volumes/workspace/default/tutorial/cust_info/"

(df_joined.write
  .format("csv")
  .mode("overwrite")
  .save(file_path)
)

In [0]:
(df_joined
 .coalesce(1)
 .write
 .option("header", "true")
 .mode("overwrite")
 .csv("/Volumes/workspace/default/tutorial/cust_info_csv_ver")
)